In [1]:
# Colab: Mount into drive
from google.colab import drive
drive.mount("/content/drive")
#place this tutorial.ipynb in your google drive under below directories (of course you need to create these folders first!):
#/SideProjects/LLM/SgLang/
%cd '/content/drive/MyDrive/SideProjects/LLM/SgLang/EmailAgent/'

Mounted at /content/drive
/content/drive/MyDrive/SideProjects/LLM/SgLang/EmailAgent


In [ ]:
!pip install arviz
import arviz as az

In [ ]:
# ============================
# Install Required Libraries
# ============================
!pip install flask flask-cors pyngrok imaplib2

In [31]:
#sqlite initialization:
import sqlite3

def create_connection():
    """Create a connection to the SQLite database."""
    try:
        conn = sqlite3.connect('emails.db')  # This creates 'emails.db' locally
        print("Connected to SQLite database")
        return conn
    except sqlite3.Error as e:
        print(f"Error: {e}")
        return None

def init_db():
    """Create the 'emails' table if it doesn't exist."""
    conn = create_connection()
    if conn:
        cursor = conn.cursor()
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS emails (
                id TEXT PRIMARY KEY,
                subject TEXT NOT NULL,
                content TEXT,
                date_received TEXT,
                sender TEXT
            )
        """)
        conn.commit()
        cursor.close()
        conn.close()
        print("Database initialized successfully.")

# Initialize the database
init_db()


Connected to SQLite database
Database initialized successfully.


In [32]:
#Fetch Emails from IMAP and Store Them in SQLite:
import imaplib
import email
from email.header import decode_header

def store_emails_in_db(imap_host, email_user, email_pass, folder="inbox", num_emails=5):
    """Fetch emails from IMAP and store them in the SQLite database."""
    conn = create_connection()
    if conn is None:
        return

    mail = imaplib.IMAP4_SSL(imap_host)
    mail.login(email_user, email_pass)
    mail.select(folder)

    status, messages = mail.search(None, "ALL")
    email_ids = messages[0].split()[-num_emails:]

    for email_id in email_ids:
        _, msg_data = mail.fetch(email_id, "(RFC822)")
        raw_email = msg_data[0][1]
        msg = email.message_from_bytes(raw_email)

        subject, encoding = decode_header(msg["Subject"])[0]
        if isinstance(subject, bytes):
            subject = subject.decode(encoding if encoding else "utf-8")

        body = ""
        if msg.is_multipart():
            for part in msg.walk():
                if part.get_content_type() == "text/plain":
                    body = part.get_payload(decode=True).decode()
                    break
        else:
            body = msg.get_payload(decode=True).decode()

        sender = msg.get("From")
        date_received = msg.get("Date")

        cursor = conn.cursor()
        query = """
            INSERT OR IGNORE INTO emails (id, subject, content, date_received, sender)
            VALUES (?, ?, ?, ?, ?)
        """
        values = (email_id.decode(), subject, body, date_received, sender)
        cursor.execute(query, values)

    conn.commit()
    cursor.close()
    conn.close()
    mail.logout()

# Example: Call the function to store emails
store_emails_in_db("imap.gmail.com", "email.agent.sun@gmail.com", "")


Connected to SQLite database


In [38]:
#Flask Backend to Serve Data
from flask import Flask, jsonify
from flask_cors import CORS
from pyngrok import ngrok

app = Flask(__name__)
CORS(app)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/emails', methods=['GET'])
def get_emails():
    """Fetch email subjects from SQLite database."""
    try:
        conn = create_connection()
        cursor = conn.cursor()
        cursor.execute("SELECT id, subject FROM emails")
        emails = [{"id": row[0], "subject": row[1]} for row in cursor.fetchall()]
        cursor.close()
        conn.close()
        return jsonify(emails)
    except sqlite3.Error as e:
        return jsonify({"error": str(e)}), 500

@app.route('/email/<email_id>', methods=['GET'])
def get_email_content(email_id):
    """Fetch the content of a specific email from SQLite database."""
    try:
        conn = create_connection()
        cursor = conn.cursor()
        cursor.execute("SELECT subject, content FROM emails WHERE id = ?", (email_id,))
        row = cursor.fetchone()
        cursor.close()
        conn.close()
        if row:
            return jsonify({"subject": row[0], "body": row[1]})
        else:
            return jsonify({"error": "Email not found"}), 404
    except sqlite3.Error as e:
        return jsonify({"error": str(e)}), 500

# Expose Flask App via ngrok
ngrok.set_auth_token("")

if __name__ == '__main__':
    public_url = ngrok.connect(5000)
    print(f" * ngrok tunnel available at: {public_url}")
    app.run(port=5000, debug=False, threaded=False)


 * ngrok tunnel available at: NgrokTunnel: "https://9def-34-46-18-28.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [23/Oct/2024 03:16:57] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Oct/2024 03:16:57] "GET /static/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Oct/2024 03:16:57] "GET /static/main.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Oct/2024 03:16:58] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [23/Oct/2024 03:16:59] "GET /emails HTTP/1.1" 200 -


Connected to SQLite database


INFO:werkzeug:127.0.0.1 - - [23/Oct/2024 03:17:03] "GET /email/5 HTTP/1.1" 200 -


Connected to SQLite database


INFO:werkzeug:127.0.0.1 - - [23/Oct/2024 03:17:04] "GET /email/8 HTTP/1.1" 200 -


Connected to SQLite database
